In [19]:
import pandas as pd
import numpy as np

T = 1

t = 10 * 12
L = 30 * 10**3
r = 24/12/100
pmt = L * r * ((1+r)**(t)) / ((1+r)**(t)-1)
credit = {"body": [], "percent": [], "payment": [pmt] * T * 12, "rest": [L]}
for i in range(T * 12):
    credit["percent"].append(credit["rest"][-1]*r)
    credit["body"].append(max(0, pmt - credit["percent"][-1]))
    credit["rest"].append(max(0, credit["rest"][-1] - credit["body"][-1]))

In [20]:
pmt

661.4429065749895

In [21]:
cost0 = [0, 1875, 1875, 1500, 750, 750, 750, 750, 750, 750, 750, 1125]
cost = [1875, 1875, 1875, 1500, 750, 750, 750, 750, 750, 750, 750, 1125] * (T-1) + cost0
months = ["16.Jun", "16.Jul", "16.Aug", "16.Sep", "16.Oct", "16.Nov", "16.Dec", "16.Jan", "16.Feb", "16.Mar", "16.Apr", "16.May"]

months_col = []
for i in range(1, T+1):
    for mnt in months:
        months_col.append(mnt + str(i))

df = {
    "Месяц":  months_col,
    "Выручка от продажи кофе": [None]*len(cost),
    "Себестоимость: зерна, молоко, сиропы": cost,
    "Валовая прибыль": [None]*len(cost),
    "Заработная плата баристы": [None]*len(cost),
    "Заработная плата менеджера": [250] * len(cost),
    "Коммунальные услуги": [30] * len(cost),
    "Проценты к уплате": [0] * len(cost),
    "Прибыль до уплаты налогов": [None]*len(cost),
    "Налог на прибыль": [None]*len(cost),
    "Чистая прибыль (NIAT)": [None]*len(cost)
}
data = pd.DataFrame(df)

data["Выручка от продажи кофе"] = data["Себестоимость: зерна, молоко, сиропы"] * 2.5
data["Валовая прибыль"] = data["Выручка от продажи кофе"] - data["Себестоимость: зерна, молоко, сиропы"]

data["Заработная плата баристы"] = 50 + 0.05 * data["Выручка от продажи кофе"]

In [22]:
pmt

661.4429065749895

In [23]:
credit["rest"][:12]

[30000,
 29938.55709342501,
 29875.88532871852,
 29811.9601287179,
 29746.756424717267,
 29680.248646636624,
 29612.410712994366,
 29543.216020679265,
 29472.63743451786,
 29400.647276633226,
 29327.2173155909,
 29252.318755327728]

In [24]:
data["Проценты к уплате"] = credit["percent"]

In [25]:
data["Прибыль до уплаты налогов"] = data["Валовая прибыль"] - data["Заработная плата баристы"] - data["Заработная плата менеджера"] - data["Коммунальные услуги"] - data["Проценты к уплате"]
data["Налог на прибыль"] = (data["Прибыль до уплаты налогов"] * 0.2).clip(0, np.inf)
data["Чистая прибыль (NIAT)"] = data["Прибыль до уплаты налогов"] - data["Налог на прибыль"]

In [26]:
df = data.set_index('Месяц', inplace=False).transpose()
df["Итого"] = data.sum()

In [27]:
df

Месяц,16.Jun1,16.Jul1,16.Aug1,16.Sep1,16.Oct1,16.Nov1,16.Dec1,16.Jan1,16.Feb1,16.Mar1,16.Apr1,16.May1,Итого
Выручка от продажи кофе,0.0,4687.500000,4687.500000,3750.000000,1875.000000,1875.000000,1875.000000,1875.000000,1875.000000,1875.000000,1875.000000,2812.500000,29062.5
"Себестоимость: зерна, молоко, сиропы",0.0,1875.000000,1875.000000,1500.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,1125.000000,11625
Валовая прибыль,0.0,2812.500000,2812.500000,2250.000000,1125.000000,1125.000000,1125.000000,1125.000000,1125.000000,1125.000000,1125.000000,1687.500000,17437.5
Заработная плата баристы,50.0,284.375000,284.375000,237.500000,143.750000,143.750000,143.750000,143.750000,143.750000,143.750000,143.750000,190.625000,2053.125
Заработная плата менеджера,250.0,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,3000
Коммунальные услуги,30.0,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,360
Проценты к уплате,600.0,598.771142,597.517707,596.239203,594.935128,593.604973,592.248214,590.864320,589.452749,588.012946,586.544346,585.046375,7113.237103
Прибыль до уплаты налогов,-930.0,1649.353858,1650.607293,1136.260797,106.314872,107.645027,109.001786,110.385680,111.797251,113.237054,114.705654,631.828625,4911.137897
Налог на прибыль,0.0,329.870772,330.121459,227.252159,21.262974,21.529005,21.800357,22.077136,22.359450,22.647411,22.941131,126.365725,1168.227579
Чистая прибыль (NIAT),-930.0,1319.483087,1320.485835,909.008638,85.051897,86.116022,87.201429,88.308544,89.437801,90.589644,91.764523,505.462900,3742.910318
